# DEAP Library Testing
**Description:** This notebook is for testing various features of the DEAP library.

## Objective
- Set up and test DEAP library.
- Create evolutionary algorithms (e.g., genetic algorithms).
- Analyze performance.


### OneMax
#### Download and import the deap library first

In [1]:
!pip install deap

In [2]:
from deap import base
from deap import creator 
from deap import tools
print("Import success")

Import success


#### Import the OpenTTDLab library

In [3]:
from openttdlab import run_experiments, bananas_ai, local_folder, parse_savegame
import pandas as pd
import plotly.express as px

#### Import the random library for the probabilities

In [4]:
import random
print("Import success")

Import success


In [5]:
# FitnessMax is a function that inherits base.Fitness (which stores the individual's fitness values) determines if we are trying to maximize (1)
# or minimize (-1) the value depending on the values passed on to the weights.
creator.create("FitnessMax", base.Fitness, weights=(-1.0,)) 

# Creates an individual class called "Individual and instances of this type are a list.
# The fitness argument passed determines that we are trying to maximize or minimize this individual's fitness value given their associated
# value.
creator.create("Individual", list, fitness=creator.FitnessMax)

In [7]:
toolbox = base.Toolbox()

In [9]:
# Attribute generator (this represents the values within our list)
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)

toolbox.register("attr_bool", random.randint, 0, 1) 

In [17]:
#ranges should be based on the info.nut file
toolbox.register("attr_max_distance", random.randint, 100, 500) # max_distance range
toolbox.register("attr_initial_vehicles", random.randint, 1, 5) # initial_vehicles range
toolbox.register("attr_add_new_vehicle_age", random.randint, 0, 365) # add_new_vehicle_age range
toolbox.register("attr_max_age_left", random.randint, 0, 730) # max_age_left range

In [19]:
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_max_distance, toolbox.attr_initial_vehicles,
                  toolbox.attr_add_new_vehicle_age, toolbox.attr_max_age_left), n=1)

### From the original DEAP library

In [34]:
# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
#                         essentially an individual would have 100 elements
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 100)

In [21]:
# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [23]:
# the goal ('fitness') function to be maximized or minimized
def evalOneMax(individual):
    return sum(individual),

In [25]:
def evalAI(individual):
    folderpath = 'C://Program Files (x86)//Steam//steamapps//common//OpenTTD//ai//MyNewAI'
    max_distance, initial_vehicles, add_new_vehicle_age, max_age_left = individual

    print([max_distance, initial_vehicles, add_new_vehicle_age, max_age_left])
    
    results = run_experiments(
    openttd_version='13.4',  # ... for a specific version of OpenTTD
    opengfx_version='7.1',   # ... and a specific version of OpenGFX
    experiments=(
       {
         # ... for random seeds
         'seed': seed,
         # ... running specific AIs. In this case, a single AI, with no
         # parameters, fetching it from https://bananas.openttd.org/package/ai
         'ais': (
             local_folder(folderpath, 'MyNewAI', ai_params=(('max_distance', int(max_distance)), ('initial_vehicles', int(initial_vehicles)), 
                                                            ('add_new_vehicle_age', int(add_new_vehicle_age)), ('max_age_left', int(max_age_left)))),
         ),
         # ... each for a number of (in game) days
         'days': 365 * 4 + 1,
       }
       for seed in range(0, 10)
        ),
    )
    # Get all the values at the end of the simulation (looking for long-term stability across as many scenarios as possible)
    filtered_results = [row for row in results if row.get('date') == results[len(results) - 1]['date']]

    aggregate_fitness = 0
    for row in filtered_results:
        total_wait_time = sum(item['wait_time'] for item in row['chunks']['ORDR'].values())
        aggregate_fitness += (total_wait_time/len(row['chunks']['VEHS']))

    aggregate_fitness = aggregate_fitness / len(filtered_results)
    print("aggregate_fitness: " + str(aggregate_fitness))
    print()
    return aggregate_fitness,

In [27]:
# register the goal / fitness function
toolbox.register("evaluate", evalAI)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.2
# meaning 1 element in the list will be mutated (doesn't automatically mean the value will change)
# Important to note that some of the values in the genome are zero even though the range is not inclusive of 0
# This may be caused by the mutation (would I need to clamp values?)
toolbox.register("mutate", tools.mutPolynomialBounded, low=[100, 1, 0, 0], up=[500, 5, 365, 730], eta=0.5, indpb=0.5)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
# essentially a 1v1v1 and the winner goes to the next generation
toolbox.register("select", tools.selTournament, tournsize=2)

In [31]:
def main():
    random.seed(64)
    generation_best_result = []
    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=10)

    # CXPB  is the probability with which two individuals
    #       are crossed (50% of the population will undergo CX)
    #
    # MUTPB is the probability for mutating an individual (20% of the population will undergo Mutation)
    CXPB, MUTPB = 0.5, 0.2

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    # if the all of the list is yet to have all elements as 1 
    # and the number of generations is still less than 100
    while g < 20:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        print() # for some extra padding

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    generation_best_result.append(best_ind)
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

if __name__ == "__main__":
    main()

Start of evolution
[343, 1, 322, 627]


Output()

aggregate_fitness: 71.4440725940726

[306, 5, 351, 17]


Output()

aggregate_fitness: 71.54498168498168

[204, 3, 360, 647]


...

aggregate_fitness: 71.4440725940726

[153, 5, 23, 603]


Output()

aggregate_fitness: 81.12605392090344

[235, 2, 276, 201]


Output()

KeyboardInterrupt: 